In [7]:
import dace
import os
import dace.library
import dace.properties
import dace.sdfg.nodes
from dace.transformation.transformation import ExpandTransformation
from dace import SDFG, SDFGState, symbolic, dtypes
import numpy as np
from dace.config import Config
import ctypes.util, os,sys
from ctypes import util
from ctypes.util import find_library
print('Python:', sys.version.split()[0])
cp = os.environ.get('CONDA_PREFIX')
print(' CONDA prefix:', cp)
lib = os.path.join(cp, 'lib')

current_conf = Config._config
print(current_conf)
default_build_folder_config = current_conf['default_build_folder']
default_build_folder_config


Python: 3.12.11
 CONDA prefix: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev
{'debugprint': 'verbose', 'frontend': {'cache_size': 0, 'implicit_recursion_depth': 64, 'raise_nested_parsing_errors': False, 'verbose_errors': False, 'preprocessing_passes': 5, 'dont_fuse_callbacks': False, 'typed_callbacks_only': False, 'unroll_threshold': -1, 'check_args': False, 'avoid_wcr': False}, 'compiler': {'build_type': 'Debug', 'codegen_lineinfo': True, 'extra_cmake_args': '-DBLA_VENDOR=OpenBLAS -DCMAKE_FIND_FRAMEWORK=LAST -DCMAKE_INCLUDE_PATH=${CONDA_PREFIX}/include -DCMAKE_LIBRARY_PATH=${CONDA_PREFIX}/lib', 'cpu': {'executable': '', 'args': '-std=c++14 -fPIC -Wall -Wextra -O3 -march=native -fno-strict-aliasing -Wno-unused-parameter -Wno-unused-label -I/opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/include', 'openmp_sections': False, 'libs': ''}, 'linker': {'args': '', 'executable': ''}, 'cuda': {'default_block_size': '64,8,1', 'backend': 'auto', 'path': '', 'args': '-Xcompiler -march=na

'.dacecache'

In [3]:

from dace import libraries
dir(libraries)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [ ]:

blas_lib = libraries.get_library('blas')
print("Default BLAS implementation:", blas.default_implementation)



Default BLAS implementation: None


In [51]:
print(f"Intel MKL installed: {blas.IntelMKL.is_installed()}; IntelMKLSparse: {blas.IntelMKLSparse.is_installed()}")
print(f"OpenBLAS installed: {blas.OpenBLAS.is_installed()}")
print(f"cuBLAS installed: {blas.cuBLAS.is_installed()}")
print(f"BLAS installed: {blas.BLAS.is_installed()}")


print(f'\nInstr. rocBLAS: {blas.rocBLAS.is_installed()}')

AttributeError: module 'dace.libraries.blas' has no attribute 'IntelMKLSparse'

In [52]:
current_conf

{'debugprint': 'verbose',
 'frontend': {'cache_size': 0,
  'implicit_recursion_depth': 64,
  'raise_nested_parsing_errors': False,
  'verbose_errors': False,
  'preprocessing_passes': 5,
  'dont_fuse_callbacks': False,
  'typed_callbacks_only': False,
  'unroll_threshold': 11,
  'check_args': False,
  'avoid_wcr': False},
 'compiler': {'build_type': 'Debug',
  'codegen_lineinfo': True,
  'extra_cmake_args': '-DBLA_VENDOR=OpenBLAS -DCMAKE_FIND_FRAMEWORK=LAST -DCMAKE_INCLUDE_PATH=${CONDA_PREFIX}/include -DCMAKE_LIBRARY_PATH=${CONDA_PREFIX}/lib',
  'cpu': {'executable': '',
   'args': '-std=c++14 -fPIC -Wall -Wextra -O3 -march=native -fno-strict-aliasing -Wno-unused-parameter -Wno-unused-label -I/opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/include',
   'openmp_sections': False,
   'libs': ''},
  'linker': {'args': '', 'executable': ''},
  'cuda': {'default_block_size': '64,8,1',
   'backend': 'auto',
   'path': '',
   'args': '-Xcompiler -march=native --use_fast_math -Xcompiler -Wn

## Using this wheel in another environment

Here are the exact checks and overrides you likely want when installing the wheel in a fresh environment that may have its own wheels/libs already:



1) Install the wheel and BLAS/LAPACK runtime

- If using conda:

  - `conda install libblas libcblas liblapack openblas` (or `mkl` on x86 if you prefer MKL)

- Then install the built wheel:

  - `pip install /path/to/dist/dace-1.0.0-py3-none-any.whl`



2) Verify BLAS/LAPACK are discoverable (like cell 14)

- This lists key libraries and shows what `ctypes.find_library` resolves:


In [77]:

print('\nctypes.find_library...')

for name in ['blas','cblas','lapack','openblas']:
    path = find_library(name)
    print(f'  {name}:', path,type(path))
print(str(ctypes))
# print(path)


ctypes.find_library...
  blas: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/../lib/libblas.dylib <class 'str'>
  cblas: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/../lib/libcblas.dylib <class 'str'>
  lapack: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/../lib/liblapack.dylib <class 'str'>
  openblas: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/../lib/libopenblas.dylib <class 'str'>
<module 'ctypes' from '/opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/lib/python3.12/ctypes/__init__.py'>


In [78]:
find_library

<function ctypes.util.find_library(name)>

In [55]:
print('Listing known BLAS/LAPACK libs under', lib)
import glob
for pat in ['libblas*','libcblas*','liblapack*','libopenblas*','libmkl*']:
    matches = sorted(glob.glob(os.path.join(lib, pat)))
    print(f'  {pat}:', len(matches))
    for m in matches[:5]:
        print('   -', os.path.basename(m))
    if len(matches) > 5:
        print('   ...')

Listing known BLAS/LAPACK libs under /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/lib
  libblas*: 3
   - libblas.3.dylib
   - libblas.dylib
   - libblas_reexport.dylib
  libcblas*: 2
   - libcblas.3.dylib
   - libcblas.dylib
  liblapack*: 3
   - liblapack.3.dylib
   - liblapack.dylib
   - liblapacke-netlib.3.9.0.dylib
  libopenblas*: 7
   - libopenblas.0.dylib
   - libopenblas.a
   - libopenblas.dylib
   - libopenblas_armv8p-r0.3.30.dylib
   - libopenblas_vortexp-r0.3.30.a
   ...
  libmkl*: 0


Two approaches to configure CMake to use specified compilers

1. Set environment variables (before starting Python)
2. Set DaCe extra CMake args programmatically

---



Option 1: set environment variables

ex. of setting CC/CXX:
```bash
  export CC="$CONDA_PREFIX/bin/clang"
  export CXX="$CONDA_PREFIX/bin/clang"
```





Option 2: set DaCe extra CMake args programmatically


In [56]:
print('extra_cmake_args:', Config.get('compiler','extra_cmake_args'))

print('extra_cmake_args:', Config.get('compiler','extra_cmake_args'))


extra_cmake_args: -DBLA_VENDOR=OpenBLAS -DCMAKE_FIND_FRAMEWORK=LAST -DCMAKE_INCLUDE_PATH=${CONDA_PREFIX}/include -DCMAKE_LIBRARY_PATH=${CONDA_PREFIX}/lib
extra_cmake_args: -DBLA_VENDOR=OpenBLAS -DCMAKE_FIND_FRAMEWORK=LAST -DCMAKE_INCLUDE_PATH=${CONDA_PREFIX}/include -DCMAKE_LIBRARY_PATH=${CONDA_PREFIX}/lib


In [57]:
compiler = Config.get('compiler')
# print('compiler:', compiler)

In [58]:
from dace.config import Config

print('Synchronous debugging enabled:', Config.get_bool('compiler', 'cuda', 'syncdebug'))

Config.set('frontend', 'unroll_threshold', value=11)

Synchronous debugging enabled: False


4) Confirm which tools are on PATH (noisy shell inits avoided)

- Use Python to find where llvm/clang live without invoking a login shell.


In [80]:
# Resolve tool locations from Python

# import shutil, os

for tool in ['cmake','llvm-config','clang','clang','mlir-opt','mlir-translate','mlir']:

    print(f'{tool}:', shutil.which(tool))


cmake: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/cmake
llvm-config: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/llvm-config
clang: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/clang
clang: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/clang
mlir-opt: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/mlir-opt
mlir-translate: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/mlir-translate
mlir: None


In [60]:

import shutil
import subprocess

tools = ['cmake', 'llvm-config', 'clang', 'clang', 'mlir-opt', 'mlir-translate', 'mlir']

def get_version(path: str) -> str | None:
    # Try common version flags
    for flag in ('--version', '-version', '-v', '-V', 'version'):
        try:
            out = subprocess.check_output([path, flag], stderr=subprocess.STDOUT, text=True, timeout=5)
            # Return first non-empty line
            for line in out.splitlines():
                s = line.strip()
                if s:
                    return s
        except Exception:
            continue
    return None

for tool in tools:
    path = shutil.which(tool)
    if path:
        ver = get_version(path)
        print(f'{tool}: {path} ({ver if ver else "version unknown"})')
    else:
        print(f'{tool}: not found')

cmake: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/cmake (cmake version 4.1.2)
llvm-config: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/llvm-config (19.1.7)
clang: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/clang (clang version 19.1.7)
clang: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/clang (clang version 19.1.7)
mlir-opt: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/mlir-opt (LLVM (http://llvm.org/):)
mlir-translate: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/mlir-translate (LLVM (http://llvm.org/):)
mlir: not found


# Probe DaCe codegen CMake variables

This notebook configures CMake against `dace/codegen` and extracts key variables using a small injected CMake script.

It reports:
- `DACE_PROGRAM_NAME`
- `DACE_LIBS`
- `HLSLIB_PART_NAME`
- `DACE_CUDA_ARCHITECTURES_DEFAULT`
- `LOCAL_CUDA_ARCHITECTURES`
- `CUDAToolkit_NVCC_EXECUTABLE`

We first try a CUDA-triggered configure (to populate CUDA-related values), then fall back to a CPU-only configure if needed.

## 1) Write dump_dace_vars.cmake probe script

We use `scripts/dump_dace_vars.cmake` to write `${CMAKE_BINARY_DIR}/dace_vars.json` during configure. It JSON-escapes values and serializes CMake lists as arrays. The fields include:
- `DACE_PROGRAM_NAME`
- `DACE_LIBS` (array)
- `HLSLIB_PART_NAME`
- `DACE_CUDA_ARCHITECTURES_DEFAULT`
- `LOCAL_CUDA_ARCHITECTURES` (array)
- `CUDAToolkit_NVCC_EXECUTABLE`

The script is already created by this workspace under `scripts/dump_dace_vars.cmake`.

## 2) Create minimal CUDA trigger source tree

CMake’s `foreach(DACE_FILE ...)` logic enables CUDA when a file resides under a `cuda/` folder. We create a temporary tree `.cmake-probe/src/cuda/dummy.cpp` to trigger CUDA detection in the configure step.

In [65]:
# Create the probe source tree and dummy CUDA file
import os
from pathlib import Path

# In notebooks, __file__ is undefined; use current working directory
repo = Path.cwd()
probe_dir = repo / '.cmake-probe'
src_dir = probe_dir / 'src' / 'cuda'
src_dir.mkdir(parents=True, exist_ok=True)
(src_dir / 'dummy.cpp').write_text('// dummy\n')
print('Created', src_dir / 'dummy.cpp')

Created /Users/sophieblock/dev_packages/dace/.cmake-probe/src/cuda/dummy.cpp


## 3) Configure CMake build with probe script included

We configure against `dace/codegen` with `DACE_ENV_CMAKE_FILES` pointing to our probe script. Configuration produces `dace_vars.json` in the build folder. If CUDA fails to configure, we fall back without CUDA.

In [64]:
# Run CMake configure with the probe (try CUDA trigger, then fallback)
import json
import subprocess
from pathlib import Path

# In notebooks, __file__ is undefined; use current working directory
repo = Path.cwd()
codegen = repo / 'dace' / 'codegen'
probe_dir = repo / '.cmake-probe'
build_dir = probe_dir / 'build'
build_dir.mkdir(parents=True, exist_ok=True)
probe_cmake = repo / 'scripts' / 'dump_dace_vars.cmake'

def run(cmd):
    print('Running:', ' '.join(map(str, cmd)))
    proc = subprocess.run(cmd, cwd=repo, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(proc.stdout)
    return proc.returncode

ret = run([
    'cmake','-S', str(codegen), '-B', str(build_dir),
    f'-DDACE_SRC_DIR={probe_dir / "src"}',
    '-DDACE_FILES=cuda/dummy.cpp',
    f'-DDACE_ENV_CMAKE_FILES={probe_cmake}',
    '-DDACE_FPGA_AUTOBUILD_BITSTREAM=OFF',
])

json_file = build_dir / 'dace_vars.json'
if not json_file.exists():
    print('\nCUDA configure did not produce JSON. Retrying without CUDA...')
    ret = run([
        'cmake','-S', str(codegen), '-B', str(build_dir),
        f'-DDACE_SRC_DIR={probe_dir / "src"}',
        '-DDACE_FILES=',
        f'-DDACE_ENV_CMAKE_FILES={probe_cmake}',
        '-DDACE_FPGA_AUTOBUILD_BITSTREAM=OFF',
    ])

print('JSON path:', json_file, 'exists:', json_file.exists())

Running: cmake -S /Users/sophieblock/dev_packages/dace/dace/codegen -B /Users/sophieblock/dev_packages/dace/.cmake-probe/build -DDACE_SRC_DIR=/Users/sophieblock/dev_packages/dace/.cmake-probe/src -DDACE_FILES=cuda/dummy.cpp -DDACE_ENV_CMAKE_FILES=/Users/sophieblock/dev_packages/dace/scripts/dump_dace_vars.cmake -DDACE_FPGA_AUTOBUILD_BITSTREAM=OFF
-- The C compiler identification is Clang 19.1.7
-- The CXX compiler identification is Clang 19.1.7
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/arm64-apple-darwin20.0.0-clang - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /opt/homebrew/Caskroom/miniforge/base/envs/dace-dev/bin/arm64-apple-darwin20.0.0-clang++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile feature

## 4) Parse and display extracted variables

We read the generated `dace_vars.json` and pretty-print the values. Lists (like `DACE_LIBS` and `LOCAL_CUDA_ARCHITECTURES`) are shown explicitly.

In [66]:
# Load and display JSON
import json
from pathlib import Path

# In notebooks, __file__ is undefined; use current working directory
repo = Path.cwd()
json_file = repo / '.cmake-probe' / 'build' / 'dace_vars.json'
if not json_file.exists():
    raise FileNotFoundError(f"Expected {json_file} not found; run the previous cell.")

data = json.loads(json_file.read_text())
print('DACE_PROGRAM_NAME:', data.get('DACE_PROGRAM_NAME'))
print('HLSLIB_PART_NAME:', data.get('HLSLIB_PART_NAME'))
print('DACE_CUDA_ARCHITECTURES_DEFAULT:', data.get('DACE_CUDA_ARCHITECTURES_DEFAULT'))
print('CUDAToolkit_NVCC_EXECUTABLE:', data.get('CUDAToolkit_NVCC_EXECUTABLE'))

print('\nDACE_LIBS:')
for v in (data.get('DACE_LIBS') or []):
    print(' -', v)

print('\nLOCAL_CUDA_ARCHITECTURES:')
for v in (data.get('LOCAL_CUDA_ARCHITECTURES') or []):
    print(' -', v)

FileNotFoundError: Expected /Users/sophieblock/dev_packages/dace/.cmake-probe/build/dace_vars.json not found; run the previous cell.

## 5) Fallback path behavior

If CUDA configuration fails or is not available on this platform, the probe re-runs CMake without the CUDA trigger. In that case, CUDA-specific variables (e.g., `LOCAL_CUDA_ARCHITECTURES`, `CUDAToolkit_NVCC_EXECUTABLE`) will be null/empty, while CPU/FPGA-related defaults still appear.

## 6) Reproducible terminal scripts

- Bash: `scripts/probe_env.sh`
- PowerShell: `scripts/probe_env.ps1`

They replicate this notebook: create a dummy `cuda/dummy.cpp`, run CMake with `DACE_ENV_CMAKE_FILES` pointing to `scripts/dump_dace_vars.cmake`, and print the resulting JSON.